#### Monte Carlo Simulation

In [1]:
import numpy as np   
import pandas as pd    
import random  as rd

In [2]:
 class Die:
    '''
    PURPOSE:
    A class accepts variety of random variables associated with stochastic
    processes and rolls/runs and returns a list of outcomes
    
    ATTRIBUTES:
    Takes an array of faces 
    
    METHODS:
    __init__::Return a dataframe with faces and weights column
    change_weight:: Changes the weight of a single side/face.
    roll_die:: Roll/Run the die one or more times
    show_state:: Display the current set of the faces and weights
    -------------------------------------------------------------------------
    '''
    def __init__(self , faces):
        '''
        PURPOSE:
        Initializes private dataframe containing faces and weights respectively
    
        INPUTS:
        Takes an array of faces 
    
        OUTPUTS:
        Return a dataframe with faces and weights column
        '''
        self.faces = list(set(faces)) # The faces must be unique
        self.weights= np.ones(len(faces))  # Initialize the weight to 1.0
        self.faces_weights_df= pd.DataFrame(self.faces , columns=['faces'])
        self.faces_weights_df = self.faces_weights_df.assign(weights=self.weights)
    def change_weight(self , face_value , new_weight):
        # face passed must be valid
        is_face_valid = (lambda face_value: face_value in self.faces) 
        if(not is_face_valid(face_value)): return "Error:The face passed is invalid."
        # Weight passed must be valid
        is_weight_valid = isinstance(new_weight, float) | isinstance(new_weight, int) | isinstance(new_weight, bool)
        if(not is_weight_valid): return "Error:The Weight passed is invalid."
        self.faces_weights_df.iloc[self.faces.index(face_value) , [1]] = new_weight
    def roll_die(self , number_of_rolls = 1):
         return [rd.choice(self.faces) for roll in range(number_of_rolls)]
    def show_state(self):
        return self.faces_weights_df

In [3]:
number_die_array = [1, 2 ,3 ,4 ,5 ,6]
string_coin_array = ['Heads', 'Tails']
d = s = Die(number_die_array)
d.show_state()
d.change_weight(2, 2.0)
d.change_weight(3, 3)
d.change_weight(4, 4)
d.change_weight(5, 5)
d.change_weight(6, 6)
d.roll_die(10)

[1, 5, 3, 4, 2, 6, 5, 6, 3, 1]

In [4]:
Die.__doc__

'\n   PURPOSE:\n   A class accepts variety of random variables associated with stochastic\n   processes and rolls/runs and returns a list of outcomes\n   \n   ATTRIBUTES:\n   Takes an array of faces \n   \n   METHODS:\n   __init__::Return a dataframe with faces and weights column\n   change_weight:: Changes the weight of a single side/face.\n   roll_die:: Roll/Run the die one or more times\n   show_state:: Display the current set of the faces and weights\n   -------------------------------------------------------------------------\n   '

In [5]:
class Game(): # Or access using Game(Die)
    def __init__(self , dice):
        self.dice = dice[0]  # Taking just one dice for now
    def play(self, number_of_rolls):
        cols = [str(die) for die in self.dice.faces]
        roll_results = [roll for roll in self.dice.roll_die(number_of_rolls)] # Roll the dice X number of times extract the results  
        # Create a single dim array with similar results populate across the array
        roll_data = np.array([np.full((1 ,len(self.dice.faces)), roll_result)[0] for roll_result in roll_results]) 
        index = [i for i in range(1, len(roll_data) + 1)]
        play_df = pd.DataFrame(roll_data ,
                               index = index,
                               columns = cols)  
        play_df.index.name = 'roll number' 
        self.show(play_df , 1)
        
    
    def show(self , play_result_df , df_form = 1): # default df return form is Wide = 1 and Narrow option is 2
        if (df_form != 1 and df_form != 2):
            return print("Error:The dataframe display format option can only be Wide(value 1) or Narrow(value 2).")
        if df_form == 1:
            print(play_result_df)
        else:
            print(pd.melt(panda_df, value_vars=cols, var_name='VARS', value_name='VALUES', ignore_index=False))
        
dice = []
dice.extend([s , d])
g = Game(dice)
g.play(10)

             1  2  3  4  5  6
roll number                  
1            6  6  6  6  6  6
2            1  1  1  1  1  1
3            3  3  3  3  3  3
4            2  2  2  2  2  2
5            4  4  4  4  4  4
6            6  6  6  6  6  6
7            3  3  3  3  3  3
8            5  5  5  5  5  5
9            2  2  2  2  2  2
10           4  4  4  4  4  4
